In [0]:
!pip install transformers -q

In [0]:
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_data_all.csv
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_answers_all.csv

In [0]:
import pandas as pd
import functools
df_data = pd.read_csv('taskA_trial_data.csv', error_bad_lines=False)
df_answers = pd.read_csv('taskA_trial_answer.csv', error_bad_lines=False, header=None, names=['id', 'labels'])

df = df_data.set_index('id').join(df_answers.set_index('id'))

In [26]:
for i in range(1, len(df)):
  if df['sent0'][i][-1] != '.' or df['sent1'][i][-1] !='.' :
    df['sent0'][i] = df['sent0'][i] +'.'
    df['sent1'][i] = df['sent1'][i] +'.'

df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,sent0,sent1,labels
id,,,
1,he put an elephant into the fridge.,he put a turkey into the fridge.,0
2,my sister eats an apple after breakfast every ...,my sister eats a stone after breakfast every day.,1
3,money can be used for buying cars.,money can be used for buying stars.,1
4,New York is located in the northeastern part o...,USA is located in the northeastern part of New...,1
5,a man can better see stars and the moon in day...,a man can hardly see stars and the moon in day...,0
...,...,...,...
2017,she cleans her teeth with toothpaste.,she cleans her teeth with shampoo.,1
2018,you can see a dinosaur in the museum.,you can see a dinosaur in the zoo.,1
2019,people have to hold onto their hats because of...,people have to hold onto their shoes because o...,1


In [27]:
df['pred'] = 0
df.head(5) 

,sent0,sent1,labels,pred
id,,,,
1,he put an elephant into the fridge.,he put a turkey into the fridge.,0,0
2,my sister eats an apple after breakfast every ...,my sister eats a stone after breakfast every day.,1,0
3,money can be used for buying cars.,money can be used for buying stars.,1,0
4,New York is located in the northeastern part o...,USA is located in the northeastern part of New...,1,0
5,a man can better see stars and the moon in day...,a man can hardly see stars and the moon in day...,0,0


In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Use a [CLS] as the classfication and train a classifier
# mask each word and get the probability of that word happening. Multiply this for all the word in the sentence. The bigger is the most probable answer

In [29]:
# Load pre-trained model (weights)
# model_type = 'bert-large-cased'
model_type = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(model_type)
tokenizer = BertTokenizer.from_pretrained(model_type)
_ = model.eval()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_siz

In [0]:
MASK_TOKEN = '[MASK]'
device = 'cuda'
# device = 'cpu'

#non-efficient but I merged two repeatitive for loop

In [0]:
for i in range(len(df)):
  if i % 500 == 0:
    print('i = ', i)
  sent0 = df['sent0'][i]    
  sent1 = df['sent1'][i]
  sentences = [sent0 , sent1]

  # Tokenization
  marked_sent0 = '[CLS] ' + sent0.replace('.' , '[SEP]')
  marked_sent1 = '[CLS] ' + sent1.replace('.' , '[SEP]')
  marked_sentences = [marked_sent0 , marked_sent1]
  
  # Tokenize sentences with the BERT tokenizer
  tokenized_sent0 = tokenizer.tokenize(marked_sent0)
  tokenized_sent1 = tokenizer.tokenize(marked_sent1)
  list_of_tokenized_sent = [tokenized_sent0 , tokenized_sent1]

  #Replace each word sent0 by MASK_TOKEN
  k = 0
  production = 1
  list_of_multiplication_of_prbblty = []
  for sent in list_of_tokenized_sent:
    for j in range(len(sent)):
      masked_token = sent[j]
      sent[j] = MASK_TOKEN
     
      #Map the token strings to their vocabulary indeces.
      indexed_tokens_sent = [tokenizer.convert_tokens_to_ids(sent)]

      #Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor(indexed_tokens_sent).to(device) # token_indices
      segments_tensors = torch.tensor([[0] * len(list_of_tokenized_sent[k])]).to(device) # segment
      model.to(device)
      
      #Predict all tokens
      with torch.no_grad():
          outputs_sent = model(tokens_tensor, token_type_ids=segments_tensors) 
          #outputs_sent[0] is a tuple # batch_size * seq_len * vocab_size eg: torch.Size([1, 9, 30522])
          list_predicted_prbblty = outputs_sent[0][0, j] #list_predicted_prbblty.shape ---> torch.Size([30522])*****tensor([-6.5220, -6.5080, -6.5015,  ..., -5.9197, -5.6961, -3.9317],device='cuda:0'  
          normalize_predicted_prbblty = torch.softmax(list_predicted_prbblty, -1)
          masked_token_index_in_vocab = tokenizer.convert_tokens_to_ids(masked_token)
          prediction_masked_token = normalize_predicted_prbblty[masked_token_index_in_vocab]  # prbblty of masked token for example ---> tensor(5.4704e-07, device='cuda:0')
          #masked_token_index_in_vocab => index of the word that is masked
          #Multiplication of the probabilities of masked tokens
      production *= prediction_masked_token.item()
      #Reset tokenized sentence to the first version to mask net word
      sent = tokenizer.tokenize(marked_sentences[k])

    #normalize_prbblty_production = pow( production , 1/len(sent))
    list_of_multiplication_of_prbblty.append(production)
    production = 1
    k +=1

#Updating pred column in the dataframe
  if list_of_multiplication_of_prbblty[0] > list_of_multiplication_of_prbblty[1]:
    df['pred'][i] = 1
  elif list_of_multiplication_of_prbblty[0] < list_of_multiplication_of_prbblty[1]:
    df['pred'][i] = 0

In [0]:
df

,sent0,sent1,labels,pred
id,,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0,0
1,He drinks apple.,He drinks milk.,0,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1,1
3,A mosquito stings me,I sting a mosquito,1,0
4,A niece is a person.,A giraffe is a person.,1,0
...,...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1,1


In [0]:
(df['labels'] == df['pred']).sum()/len(df)

0.6976

### Don't repeat yourself:
prod = 1
for ....
  prod *= prop.item()

### combine all input for a model inside just one tensor. batch_size = seq_len

#Efficient2- insted of batch size = the lenth of sequense

In [33]:
for i in range(1,len(df)):
  if i % 100 == 0:
    print('i = ', i)
  sent0 = df['sent0'][i]    
  sent1 = df['sent1'][i]
  sentences = [sent0 , sent1]

  # add [CLS] and [SEP] to the beginning and end of the sentences
  marked_sent0 = '[CLS] ' + sent0.replace('.' , '[SEP]')
  marked_sent1 = '[CLS] ' + sent1.replace('.' , '[SEP]')
  marked_sentences = [marked_sent0 , marked_sent1]

  # Tokenize sentences with the BERT tokenizer
  tokenized_sent0 = tokenizer.tokenize(marked_sent0)
  tokenized_sent1 = tokenizer.tokenize(marked_sent1)
  list_of_tokenized_sent = [tokenized_sent0 , tokenized_sent1]

  k = 0 
  list_of_indices_tensors = []
  merged_tensor = []
  segment = []
  merged_segment = []
  for sent in list_of_tokenized_sent:
    for j in range(len(sent)):
      masked_token = sent[j]
      sent[j] = MASK_TOKEN
      #Map the token strings to their vocabulary indeces.
      indexed_tokens_sent = tokenizer.convert_tokens_to_ids(sent)
      #Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor(indexed_tokens_sent).to(device) # token_indices
      list_of_indices_tensors.append(tokens_tensor)
      segments_tensors = torch.tensor([0] * len(list_of_tokenized_sent[k])).to(device) # segment
      segment.append(segments_tensors) 
      sent = tokenizer.tokenize(marked_sentences[k])    

    merge_all_tensors = torch.stack(list_of_indices_tensors)
    merge_all_segment = torch.stack(segment)
    #the following tensor is the first argument of the model including index tensors of all these:
    #[['[MASK]', 'he', 'poured', 'orange', 'juice', 'on', 'his', 'cereal', '[SEP]'],
    #['[CLS]', '[MASK]', 'poured', 'orange', 'juice', 'on', 'his', 'cereal', '[SEP]'], 
    #['[CLS]', 'he', '[MASK]', 'orange', 'juice', 'on', 'his', 'cereal', '[SEP]'],
    #['[CLS]', 'he', 'poured', '[MASK]', 'juice', 'on', 'his', 'cereal', '[SEP]'], 
    #['[CLS]', 'he', 'poured', 'orange', '[MASK]', 'on', 'his', 'cereal', '[SEP]'],
    #['[CLS]', 'he', 'poured', 'orange', 'juice', '[MASK]', 'his', 'cereal', '[SEP]'], 
    #['[CLS]', 'he', 'poured', 'orange', 'juice', 'on', '[MASK]', 'cereal', '[SEP]'], 
    #['[CLS]', 'he', 'poured', 'orange', 'juice', 'on', 'his', '[MASK]', '[SEP]'], 
    #['[CLS]', 'he', 'poured', 'orange', 'juice', 'on', 'his', 'cereal', '[MASK]']]
    merged_tensor.append(merge_all_tensors)
    
    merged_segment.append(merge_all_segment)
    model.to(device)
    list_of_indices_tensors = []
    segment = []
    k += 1

  #Calling BERT model
  with torch.no_grad():
    outputs_sent0 = model(merged_tensor[0], token_type_ids=merged_segment[0])
    outputs_sent1 = model(merged_tensor[1], token_type_ids=merged_segment[1])
    list_output = [outputs_sent0 , outputs_sent1]
    #outputs_sent0[0] is a tuple of batch size * seq length * vocab
    #in our case is j*j*30522
  

  #predicting masked_token prbblty
  element = 0
  production = 1
  list_of_multiplication_of_prbblty = []
  for sent in list_of_tokenized_sent:
    for j in range(len(sent)):
      masked_token = sent[j]
      sent[j] = MASK_TOKEN
      #normalized prbblty of all tokens in dictionary.--------> list_output[k][0][***********]---------> torch.Size([30522])
      normalize_predicted_prbblty = torch.softmax(list_output[element][0][j, j, :], -1) 
      #print(normalize_predicted_prbblty)
      #find index of masked_token
      masked_token_index_in_vocab = tokenizer.convert_tokens_to_ids(masked_token)
      #print(masked_token_index_in_vocab)
      #probblty of masked_token
      prediction_masked_token = normalize_predicted_prbblty[masked_token_index_in_vocab]
      #print(prediction_masked_token)
      production *= prediction_masked_token.item()
      sent = tokenizer.tokenize(marked_sentences[element])
    
    #normalize_prbblty_production = pow( production , 1/len(sent))
    list_of_multiplication_of_prbblty.append(production)
    production = 1
    element +=1


  #Updating pred column in the dataframe
  if list_of_multiplication_of_prbblty[0] > list_of_multiplication_of_prbblty[1]:
    df['pred'][i] = 1
  elif list_of_multiplication_of_prbblty[0] < list_of_multiplication_of_prbblty[1]:
    df['pred'][i] = 0
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


i =  100
i =  200
i =  300
i =  400
i =  500
i =  600
i =  700
i =  800
i =  900
i =  1000
i =  1100
i =  1200
i =  1300
i =  1400
i =  1500
i =  1600
i =  1700
i =  1800
i =  1900
i =  2000


In [34]:
df

,sent0,sent1,labels,pred
id,,,,
1,he put an elephant into the fridge.,he put a turkey into the fridge.,0,0
2,my sister eats an apple after breakfast every ...,my sister eats a stone after breakfast every day.,1,1
3,money can be used for buying cars.,money can be used for buying stars.,1,1
4,New York is located in the northeastern part o...,USA is located in the northeastern part of New...,1,1
5,a man can better see stars and the moon in day...,a man can hardly see stars and the moon in day...,0,0
...,...,...,...,...
2017,she cleans her teeth with toothpaste.,she cleans her teeth with shampoo.,1,1
2018,you can see a dinosaur in the museum.,you can see a dinosaur in the zoo.,1,0
2019,people have to hold onto their hats because of...,people have to hold onto their shoes because o...,1,0


In [35]:
(df['labels'] == df['pred']).sum()/len(df)

0.7021276595744681

In [40]:
df.drop(columns=['sent0', 'sent1' , 'labels'])

,pred
id,
1,0
2,1
3,1
4,1
5,0
...,...
2017,1
2018,0
2019,0


In [0]:
df.to_csv('subtaskA_trial_data.csv', header=False, columns=['pred'])